In [1]:
# set dependencies
import os
import csv
import requests
import time
import psycopg2
import pymongo

import pandas as pd

from flask_pymongo import PyMongo
from splinter import Browser
from bs4 import BeautifulSoup as bs
from flask import Flask, render_template, redirect


pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows=50

In [2]:
# Set up splinter function
def activate_driver():
    executable_path = {'executable_path': 'Resources/chromedriver.exe'}
    return Browser('chrome', **executable_path, headless=False)

# Nasa Mars News

In [3]:
# Collect latest news title and text from NASA mars news
browser=activate_driver()
browser.visit('https://mars.nasa.gov/news/')
html=browser.html
soup=bs(html, 'html.parser')

# Get data
news=soup.find_all('div', class_='list_text')[0]
title=news.find(class_='content_title').text
body=news.find(class_='article_teaser_body').text
date=news.find(class_='list_date').text

# Create dictionary
mars_data= {
    'Title': title,
    'Body' : body,
    'Date': date
}
          
# Close browser
browser.quit()

In [4]:
print(mars_data['Date'])
print('---------------------------------------------------------')
print(mars_data['Title'])
print('---------------------------------------------------------')
print(mars_data['Body'])

May 20, 2021
---------------------------------------------------------
Salts Could Be Important Piece of Martian Organic Puzzle
---------------------------------------------------------
A NASA team has found that organic, or carbon-containing, salts are likely present on Mars, with implications for the Red Planet's past habitability.


# JPL Mars Space Images - Featured Image

In [5]:
# Create Beautiful Soup object and view results
jpl_response=requests.get('https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html')
jpl_soup=bs(jpl_response.text,'html.parser')
jpl_soup

<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet"/>
<!-- <link rel="stylesheet" type="text/css" href="css/font.css"> -->
<link href="css/app.css" rel="stylesheet" type="text/css"/>
<link href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<title>Space Image</title>
</head>
<body>
<div class="header">
<nav class="navbar navbar-expand-lg">
<a class="navbar-brand" href="#"><img id="logo" src="image/nasa.png"/><span class="logo">Jet Propulsion Laboratory</span>
<span class="logo1">California Institute of Technology</span></a>
<button aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation" class="navbar-toggler" data-target="#navbarNav" data-toggle="collapse" type="button">
<span class="navbar-toggler-icon"></span>
</button>
<di

In [6]:
# Create url for featured image full size jpg
browser=activate_driver()
browser.visit('https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html')
html=browser.html
soup=bs(html,'html.parser')

featured_image=soup.find('img',class_='headerimage')
featured_image_src=featured_image['src']
featured_image_url=(f'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/{featured_image_src}')

browser.quit()

In [7]:
print(featured_image_url)

https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars2.jpg


# Mars Facts

In [8]:
# Get Mars data table
tables=pd.read_html('https://space-facts.com/mars/')
table=pd.DataFrame(tables[0])

table=table.rename(columns={
    0:'Data Point',
    1:'Value'
})

table.set_index('Data Point',inplace=True)

# Creating the html Table
table_html=table.to_html()
table_html=table_html.replace('\n','')

# Mars Hemispheres

In [13]:
# Create full resolution image URLs and create dictionary with title and url
browser=activate_driver()
browser.visit('https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')
hemisphere_image_urls=[]

for i in range (4):
    time.sleep(10)
    images=browser.find_by_tag('h3')
    images[i].click()
    html=browser.html
    soup=bs(html,'html.parser')
    partial_url=soup.find('img',class_='wide-image')['src']
    image_title=soup.find('h2',class_='title').text
    image_url=f'https://astrogeology.usgs.gov{partial_url}'
    image_dict={'title':image_title,'image_url':image_url}
    hemisphere_image_urls.append(image_dict)
    browser.back()

browser.quit()

In [11]:
# Final Dictionary of All Gathered Data
mars_data.update({
    "featured_image_url": featured_image_url,
    "table": table_html,
    "image_title_1": hemisphere_image_urls[0]["title"],
    "image_url_1": hemisphere_image_urls[0]["image_url"],
    "image_title_2": hemisphere_image_urls[1]["title"],
    "image_url_2": hemisphere_image_urls[1]["image_url"],
    "image_title_3": hemisphere_image_urls[2]["title"],
    "image_url_3": hemisphere_image_urls[2]["image_url"],
    "image_title_4": hemisphere_image_urls[3]["title"],
    "image_url_4": hemisphere_image_urls[3]["image_url"]
})

In [14]:
mars_data

{'Title': 'Salts Could Be Important Piece of Martian Organic Puzzle',
 'Body': "A NASA team has found that organic, or carbon-containing, salts are likely present on Mars, with implications for the Red Planet's past habitability.",
 'Date': 'May 20, 2021',
 'featured_image_url': 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars2.jpg',
 'table': '<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Data Point</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th> 